In [2]:
import nibabel as nb
import numpy as np
import scipy.io
import scipy
from scipy import stats
import math

# Reading movie watching data

In [4]:
left_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']

In [5]:
left_data.shape
right_data.shape

(32492, 3655)

The average data activity of all subjects in the 3655 time points,
32492 is number of vertices

# Reading ROI datas and geting the mean

## Left ROI

In [7]:
left_ROI_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
left_ROI = nb.load(left_ROI_path)
d_arr_left_ROI = left_ROI.darrays
left_ROI = d_arr_left_ROI[0].data
print("left ROI:",left_ROI)
left_ROI_indexes = [index for index, element in enumerate(left_ROI) if element != 0]
# print(left_ROI[left_ROI_indexes[0]])
print("number of left ROI:",len(left_ROI_indexes))

left ROI: [0. 0. 0. ... 0. 0. 0.]
number of left ROI: 226


After extracting the left ROIs indices we want to get the sum of the activity of indexes in 3655 time points

In [8]:
LeftROIsData = np.array(left_data[left_ROI_indexes])

In [9]:
LeftROIsData.shape

(226, 3655)

## Right ROI

In [10]:
right_ROI_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
right_ROI = nb.load(right_ROI_path)
d_arr_right_ROI = right_ROI.darrays
right_ROI = d_arr_right_ROI[0].data
print("right ROI:",right_ROI)
right_ROI_indexes = [index for index, element in enumerate(right_ROI) if element != 0]
print("number of right ROI:",len(right_ROI_indexes))

right ROI: [0. 0. 0. ... 0. 0. 0.]
number of right ROI: 341


In [11]:
RightROIsData = np.array(right_data[right_ROI_indexes])

In [12]:
RightROIsData.shape

(341, 3655)

# geting the mean of the left and right ROIS

In [13]:
ROIsData = np.concatenate((LeftROIsData, RightROIsData))

In [14]:
ROIsData.shape

(567, 3655)

In [15]:
MeanActivityROIs = np.mean(ROIsData, axis=0)

In [16]:
MeanActivityROIs

array([-0.48533266, -0.54377275, -0.50547335, ..., -0.61457157,
       -0.6044903 , -0.55319973])

# Correlation between mean activity ROIs in movie watching data and each vertex in left hemisphere

In [17]:
corr_left = np.zeros(32492)
pvalue_left = np.zeros(32492)
for i in range (32492):
    corr,p_value = stats.pearsonr(MeanActivityROIs, left_data[i])
    corr_left[i] = corr
    pvalue_left[i] = p_value

C:\Users\roza\AppData\Local\Temp\ipykernel_792\3710387897.py:4: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(MeanActivityROIs, left_data[i])


In [18]:
corr_left = np.nan_to_num(corr_left)
pvalue_left = np.nan_to_num(pvalue_left)

In [19]:
corr_left_32 = corr_left.astype(np.float32)
pvalue_left_32 = pvalue_left.astype(np.float32)

saving the correlation map

In [20]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = corr_left_32
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/correlation_classic_place_patches_with_all_vertices/correlation_classic_place_patches_with_left_vertices.L.func.gii')

# Significance of correlation Left

there are two types of 0 in pvalue:
1- Nan pvalues that are convert to 0
2- pvalues for correlations near 1 so the pvalue is lower than smallest number that we have, we replace this zeros with the smallest float number

In [26]:
min_float64 = np.finfo(np.float64).tiny

In [27]:
for index,pval in enumerate(pvalue_left):
    if pval == 0 and corr_left[index] != 0:
        pvalue_left[index] = min_float64

In [28]:
significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)

C:\Users\roza\AppData\Local\Temp\ipykernel_792\2113548353.py:1: RuntimeWarning: divide by zero encountered in log
  significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)
C:\Users\roza\AppData\Local\Temp\ipykernel_792\2113548353.py:1: RuntimeWarning: invalid value encountered in multiply
  significance_left = -1 * np.log(pvalue_left) * np.sign(corr_left)


In [29]:
significance_left = np.nan_to_num(significance_left, nan=0.0, posinf=0.0, neginf=0.0)

In [30]:
significance_left_32 = significance_left.astype(np.float32)

In [31]:
temp_l = nb.load('E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii')
temp_l.darrays[0].data = significance_left_32
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/correlation_classic_place_patches_with_all_vertices/significance_correlation_classic_place_patches_with_left_vertices.L.func.gii')

# Correlation between mean activity ROIs in movie watching data and each vertex in right hemisphere

In [32]:
corr_right = np.zeros(32492)
pvalue_right = np.zeros(32492)
for i in range (32492):
    corr,p_value = stats.pearsonr(MeanActivityROIs, right_data[i])
    corr_right[i] = corr
    pvalue_right[i] = p_value

C:\Users\roza\AppData\Local\Temp\ipykernel_792\2099817572.py:4: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr,p_value = stats.pearsonr(MeanActivityROIs, right_data[i])


In [33]:
corr_right = np.nan_to_num(corr_right)
pvalue_right = np.nan_to_num(pvalue_right)

In [34]:
corr_right_32 = corr_right.astype(np.float32)
pvalue_right_32 = pvalue_right.astype(np.float32)

In [35]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = corr_right_32
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/correlation_classic_place_patches_with_all_vertices/correlation_classic_place_patches_with_right_vertices.R.func.gii')

# Significance of correlation Right

In [36]:
for index,pval in enumerate(pvalue_right):
    if pval == 0 and corr_right[index] != 0:
        pvalue_right[index] = min_float64
SignificanceRight = -1 * np.log(pvalue_right) * np.sign(corr_right)
SignificanceRight = np.nan_to_num(SignificanceRight, nan=0.0, posinf=0.0, neginf=0.0)
SignificanceRight_32 = SignificanceRight.astype(np.float32)

C:\Users\roza\AppData\Local\Temp\ipykernel_792\3442036093.py:4: RuntimeWarning: divide by zero encountered in log
  SignificanceRight = -1 * np.log(pvalue_right) * np.sign(corr_right)
C:\Users\roza\AppData\Local\Temp\ipykernel_792\3442036093.py:4: RuntimeWarning: invalid value encountered in multiply
  SignificanceRight = -1 * np.log(pvalue_right) * np.sign(corr_right)


In [37]:
temp_r = nb.load('E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii')
temp_r.darrays[0].data = SignificanceRight_32
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/correlation_classic_place_patches_with_all_vertices/significance_correlation_classic_place_patches_with_right_vertices.R.func.gii')